In [1]:
import os, pyodbc
import pandas as pd
#import numpy as np
from pandas import DataFrame
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from sqlalchemy import create_engine


In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\python27\arcgis10.2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Connect to SQL db and create dataframe

In [3]:
engine = create_engine("mssql+pyodbc://gt_KL_Master")


Query for Au and Cu standards

In [4]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', 'Au' as 'Element', c.Reference as 'STD', \
c.Col101 as 'val', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and c.Col101 is not NULL \
and exists (Select * from tblTypeStandard f where c.Reference = f.Abreviation and f.Element = 'Col101') \
and  e.DateEmis >= DATEADD(MONTH, -3, GETDATE()) and c.Reference <> 'CM24'  and isnumeric(Col101) = 1\
\
UNION \
\
Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', 'Cu' as 'Element', c.Reference as 'STD', \
COALESCE(CAST(c.Col99 as float), CAST(c.Col98 as float)/10000) as 'val', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and (c.Col98 is not NULL or c.Col99 is not NULL)\
and e.DateEmis >= DATEADD(MONTH, -3, GETDATE()) and exists (Select * from tblTypeStandard  f where   f.Abreviation = c.Reference \
and f.Element in ('Col98','Col99'))"


In [22]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', 'Au' as 'Element', c.Reference as 'STD', \
c.Col101 as 'val', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and c.Col101 is not NULL \
and exists (Select * from tblTypeStandard f where c.Reference = f.Abreviation and f.Element = 'Col101') \
and e.DateEmis BETWEEN '2017-07-01' and '2017-10-01'\
and c.Reference <> 'CM24'  and isnumeric(Col101) = 1\
\
UNION \
\
Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', 'Cu' as 'Element', c.Reference as 'STD', \
COALESCE(CAST(c.Col99 as float), CAST(c.Col98 as float)/10000) as 'val', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and (c.Col98 is not NULL or c.Col99 is not NULL)\
and e.DateEmis BETWEEN '2017-07-01' and '2017-10-01'\
and exists (Select * from tblTypeStandard  f where   f.Abreviation = c.Reference \
and f.Element in ('Col98','Col99'))"


In [10]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', 'Cu' as 'Element', c.Reference as 'STD', \
COALESCE(CAST(c.Col99 as float), CAST(c.Col98 as float)/10000) as 'val', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and (c.Col98 is not NULL or c.Col99 is not NULL)\
and c.Reference = 'CuSTD2'\
and e.DateEmis <= '2017-01-01' and exists (Select * from tblTypeStandard  f where   f.Abreviation = c.Reference \
and f.Element in ('Col98','Col99'))"

In [11]:
df = pd.read_sql(sql, engine) #, index_col = 'Date')

In [12]:
df.index = df.Date

In [13]:
groups = df.groupby(['Element','STD']).mean()
graphs = groups.index.get_values()

Generate Standard Graph

In [14]:
# with plt.xkcd():
fig = plt.figure(facecolor = 'w', figsize = (10, 12), tight_layout = False)
#fig.suptitle('CRM Analyses for Q1 and Q2 of 2017', fontsize = 16)
fig.suptitle('CRM Analyses for week of %s' % (datetime.date.today() + datetime.timedelta(4 - \
               datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 16)
#fig.figimage(logo, 700,1025, zorder=1)
length = 2

for a, g in enumerate(graphs[2:]):   

    ax1 = fig.add_subplot(length,1,a+1)
    time_buffer = max((df.index.max() - df.index.min()) * 0.05, timedelta(5))
    datarange = [df.index.min() - time_buffer, df.index.max()+ time_buffer]

    #standard graphs  
    if g[0] == 'Au':
        el = "Col101"
        unit = "g/t"
    elif g[0] == 'Cu':
        el = "Col99"
        unit = "%"
    std = pd.read_sql("Select Moyenne as 'avg', Ecart_type as 'sigma' from tblTypeStandard where Element \
    = '{}' and Abreviation = '{}'".format(el, g[1]), engine)
    std_mean = std.values[0][0]
    std_sd = std.values[0][1]
    subset = df[(df.STD == g[1]) & (df.Element == g[0])]
    ax1.plot(datarange, [subset.val.mean(), \
                         subset.val.mean()], 'r-', \
              label = 'measured avg = %0.3f' % (subset.val.mean()), linewidth = 1)
    ax1.plot()
    ax1.plot(datarange, [std_mean, std_mean], 'b-', label = 'published avg = %0.3f' % (std_mean), linewidth = 1)
    _2sig1 = subset.val.mean() + 2 * subset.val.std()
    _2sig2 = subset.val.mean() - 2 * subset.val.std()
    _3sig1 = subset.val.mean() + 3 * subset.val.std()
    _3sig2 = subset.val.mean() - 3 * subset.val.std()
    _2sigm1 = std_mean + 2 * std_sd
    _2sigm2 = std_mean - 2 * std_sd
    _3sigm1 = std_mean + 3 * std_sd
    _3sigm2 = std_mean - 3 * std_sd
    ax1.plot(datarange, [_2sig1,_2sig1], 'r--', label = u'measured 2 sigma', linewidth = 1)
    ax1.plot(datarange, [_2sig2,_2sig2], 'r--')
    ax1.plot(datarange, [_3sig1,_3sig1], 'r:', label = u'meausred 3 sigma', linewidth = 1)
    ax1.plot(datarange, [_3sig2,_3sig2], 'r:')
    ax1.plot(datarange, [_2sigm1,_2sigm1], 'b--', label = u'published 2 sigma', linewidth = 1)
    ax1.plot(datarange, [_2sigm2,_2sigm2], 'b--')
    ax1.plot(datarange, [_3sigm1,_3sigm1], 'b:', label = u'published 3 sigma', linewidth = 1)
    ax1.plot(datarange, [_3sigm2,_3sigm2], 'b:', linewidth = 2)
    ax1.plot(subset.val, 'wo', label = '_nolegend_')
    ax1.plot(subset.val, 'k.', label = '_nolegend_')
    labels = subset['SAMPLEID'][(subset.val < std_mean - 2 * std_sd) |\
                          (subset.val > std_mean + 2 * std_sd)]
    for d, l in labels.iteritems():
         ax1.annotate(l, xy = (d, subset.val[subset.SAMPLEID == l]), xytext=(5, 5), textcoords= 'offset points', color = 'r')
    ylim = ax1.get_ylim()
    yshift = (ylim[1] - ylim[0]) /10.0

    ax1.set_ylim(ylim[0] - yshift, ylim[1] + yshift)
    ax1.legend(loc = 'upper left', fontsize = 'small')
    ax1.set_title('%s Analyses for %s, n= %d' % (g[0], g[1], subset.val.count()), fontsize = 12)

    ax1.set_ylabel(g[0] + " " + unit)
    xticks = ax1.get_xticks()
    date_labels = [datetime.date.fromordinal(int(f)) for f in xticks]
    ax1.set_xticklabels([])
    if a == length -1:
      ax1.set_xticklabels([x.strftime('%d-%b-%y') for x in date_labels], rotation = 45)
    
#fig.savefig('C:\\Temp\\crm.jpg', dpi = 200)  




In [22]:
df.std()

val    0.034003
dtype: float64